Please download the archive of the exercise here: https://drive.google.com/file/d/1ILlZEjCPro6PYxfbLNfm6SKSb9tPIYfL/view?usp=sharing

Unzip che archive and copy this notebook inside the extracted folder to run the exercise as it is.

In [1]:
!pip install -q tensorflow==2.0.0
!pip install adversarial-robustness-toolbox[all]
!pip install h5py==2.10.0
!pip install Pillow

# In più
!pip install numpy==1.20.3

ERROR: Could not find a version that satisfies the requirement tensorflow==2.0.0 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0)
ERROR: No matching distribution found for tensorflow==2.0.0


  Using cached h5py-2.10.0.tar.gz (301 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for h5py
Failed to build h5py
  Attempting uninstall: h5py
    Found existing installation: h5py 3.7.0
    Uninstalling h5py-3.7.0:
      Successfully uninstalled h5py-3.7.0
  Running setup.py install for h5py: started
  Running setup.py install for h5py: finished with status 'error'
  Rolling back uninstall of h5py
  Moving to c:\users\aless\appdata\roaming\python\python39\site-packages\h5py-3.7.0.dist-info\
   from C:\Users\aless\AppData\Roaming\Python\Python39\site-packages\~5py-3.7.0.dist-info
  Moving to c:\users\aless\appdata\roaming\python\python39\site-packages\h5py\
   from C:\Users\aless\AppData\Roaming\Python\Python39\site-packages\~5py


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [65 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-3.9
      creating build\lib.win-amd64-3.9\h5py
      copying h5py\h5py_warnings.py -> build\lib.win-amd64-3.9\h5py
      copying h5py\highlevel.py -> build\lib.win-amd64-3.9\h5py
      copying h5py\ipy_completer.py -> build\lib.win-amd64-3.9\h5py
      copying h5py\version.py -> build\lib.win-amd64-3.9\h5py
      copying h5py\__init__.py -> build\lib.win-amd64-3.9\h5py
      creating build\lib.win-amd64-3.9\h5py\_hl
      copying h5py\_hl\attrs.py -> build\lib.win-amd64-3.9\h5py\_hl
      copying h5py\_hl\base.py -> build\lib.win-amd64-3.9\h5py\_hl
      copying h5py\_hl\compat.py -> build\lib.win-amd64-3.9\h5py\_hl
      copying h5py\_hl\dataset.py -> build\lib.win-amd64-3.9\h5py\_hl
      copying h5py\_hl\datatype.py ->

In [1]:
import librosa

import numpy as np
import pandas as pd
import IPython.display as ipd

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

# Audio model
from model import SpeakerID

# ART
import art
from art import config
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod

C:\Users\aless\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Settings
checkpoint_path = 'resnet18_mel_25_10_norm.h5'
csv_path = 'database.csv'
file_path = 'id10001.wav'
file_id = 'id10001'

In [3]:
# Read db file list
database = pd.read_csv(csv_path)

# Compute labeles
labeler = LabelEncoder()
labeler.fit(database['VoxCeleb1 ID'])

# Get class num from id
class_num = labeler.transform([file_id])[0]
print(class_num)
# Get id from class num
print(labeler.inverse_transform([class_num])[0])

FileNotFoundError: [Errno 2] No such file or directory: '../Progetto/Progetto_temp/database.csv'

In [1]:
# Load the model
input_shape = (None, 1)
model = SpeakerID(input_shape, checkpoint_path, n_classes=1251)

NameError: name 'SpeakerID' is not defined

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 1)]         0         
_________________________________________________________________
lambda (Lambda)              (None, None, 1)           0         
_________________________________________________________________
log_melgram_layer (LogMelgra (None, None, 128)         0         
_________________________________________________________________
z_score_normalization (ZScor (None, None, 128)         0         
_________________________________________________________________
model (Model)                (None, 1251)              11822374  
Total params: 11,822,374
Trainable params: 11,814,436
Non-trainable params: 7,938
_________________________________________________________________


In [11]:
def count_mfcc(source_file_name):
    temp_wave_x, temp_sample_rate = librosa.load(source_file_name, sr =16000)
    mfccs = librosa.feature.mfcc(temp_wave_x, sr=temp_sample_rate)
    print("Total MFCC features " + str(mfccs.shape[0]) + " for the total " + str(mfccs.shape[1]) + " frames." )    

In [ ]:
# Read the audio file
x, sample_rate = librosa.load(file_path, sr=16000)
print(x, sample_rate)
# Create batch
x = np.expand_dims(x, 0)
# Add channels dimension
x = np.expand_dims(x, -1)

#Quello xhw dobbiamo fare è attaccare il classificatore
#Attacchi efficaci con la voce che rimane sempre la stessa
#Misclassificare il campione e simpatico è fare l'impresonation registrando la nostra voce e fare un attacco di impersonation
#l'anno scorso qualcuno l ha fatto
#dato il classificatore di base come viene progettata la diagnosi di sicurezza del sistema per un dato utilizzo

In [ ]:
ipd.Audio(x[0,:,0], rate=16000, autoplay=False)

In [ ]:
# Predict
y_pred = model.predict(x)

# log
print("Predicted class: ", labeler.inverse_transform(np.argmax(y_pred,axis=1))[0], " Actual class: ", file_id)
print("Prediction confidence: ", np.max(y_pred[0]))

Predicted class:  id10001  Actual class:  id10001
Prediction confidence:  0.9999651


# ART

In [ ]:
classifier = KerasClassifier(clip_values=(-1, 1), model=model, use_logits=False)

In [ ]:
classifier = KerasClassifier(clip_values=(-1, 1), model=model, use_logits=False)

In [ ]:
epsilon = 0.1

attack = FastGradientMethod(estimator=classifier, eps=epsilon, targeted=False)
x_adv = attack.generate(x)
model_predictions = model.predict(x_adv)
perturbation = np.mean(np.abs((x_adv - class_num)))

print('Average perturbation: {:4.2f}'.format(perturbation))
print("Predicted class: ", labeler.inverse_transform(np.argmax(model_predictions,axis=1))[0], " Actual class: ", file_id)
print("Prediction confidence: ", np.max(model_predictions[0]))

ipd.Audio(x_adv[0,:,0], rate=16000, autoplay=False)

Average perturbation: 0.99
Predicted class:  id10925  Actual class:  id10001
Prediction confidence:  0.94677466


Complete the exercise trying to attack the neural network with error-generic and error-specific attacks.